In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical
from keras.models import load_model
import re
import os

In [5]:
# Load your airline satisfaction dataset
airline_df = pd.read_excel("/content/BA_AirlineReviews_CL_excel.xlsx")

In [7]:
# Assuming 'OverallRating' is your target variable
df = airline_df.copy()

In [8]:
# Preprocess the text data in 'ReviewBody'
df['ReviewBody'] = df['ReviewBody'].apply(lambda x: str(x).lower())
df['ReviewBody'] = df['ReviewBody'].apply(lambda x: re.sub(r'[^\w\d ,]', '', str(x)))

In [10]:
# Assuming 'OverallRating' is your target variable
# Use 'OverallRating' as the target variable and 'ReviewBody' as features
X = df['ReviewBody']
y = df['Satisfaction']

In [11]:
# TF-IDF vectorization
tfidf = TfidfVectorizer(binary=True)
X_tfidf = tfidf.fit_transform(X).todense()

In [12]:
# Encode the target variable
lb = LabelEncoder()
y_encoded = lb.fit_transform(y)
y_categorical = to_categorical(y_encoded)

In [13]:
# Split the dataset
seed = 29
X_train, X_val, y_train, y_val = train_test_split(X_tfidf, y_categorical, test_size=0.2, random_state=seed)

In [14]:
# Build and compile the MLP model
MLP = Sequential()
MLP.add(Dense(512, input_shape=(X_tfidf.shape[1],), activation='relu'))
MLP.add(Dropout(0.5))
MLP.add(Dense(y_categorical.shape[1], activation='softmax'))
MLP.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

MLP.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               7968768   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 7973898 (30.42 MB)
Trainable params: 7973898 (30.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
# Model training
file_path = "mlp_airline.keras"
check_point = ModelCheckpoint(file_path, monitor="val_accuracy", verbose=1, save_best_only=True, mode="max")
early_stop = EarlyStopping(monitor="val_accuracy", mode="max", patience=5)

mlp_history = MLP.fit(X_train, y_train, batch_size=128, epochs=50,
                      validation_data=(X_val, y_val), callbacks=[check_point, early_stop])

Epoch 1/50
 9/24 [==========>...................] - ETA: 2s - loss: 0.0365 - accuracy: 1.0000

KeyboardInterrupt: ignored

In [19]:
# Load the best model for predictions
mlp_best = load_model(file_path)

In [20]:
# Evaluate model on validation set and print accuracy
accuracy = mlp_best.evaluate(X_val, y_val, verbose=0)[1]
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

Validation Accuracy: 34.14%


In [21]:
# Make predictions on test data
# Assuming you have a test dataset 'X_test_tfidf'
mlp_pred = mlp_best.predict(X_test_tfidf)

NameError: ignored